# Disclaimer

This can only properly be run on a heavy duty PC

# Imports and folder setup

In [38]:
from ebay_delivery_prediction_project import preprocessing, Visualisation, preprocessing_models, postprocessing
preprocessing.import_test()

from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import os

preprocessed_file_name = "preprocessed_training_data.csv"
preprocessed_data_folder = "preprocessed_data"

preprocessed_data_folder_path = os.path.join("../data", preprocessed_data_folder)
preprocessed_data_path = os.path.join(preprocessed_data_folder_path, preprocessed_file_name)

Preprocessing successfully imported.


# Run this if preprocessing isn't complete

In [39]:
# data_object = Preprocessing.read_data(rows_to_read = None, columns = ["b2c_c2c"])
# data_object = Preprocessing.read_data(rows_to_read = None)

training_rows_to_read = 15000000 # This needs to be 15000000

training_data = preprocessing.read_data(rows_to_read = training_rows_to_read)["train"]
print(training_data.columns)

Reading 15000000 rows.
Reading all columns.
Index(['b2c_c2c', 'seller_id', 'declared_handling_days',
       'acceptance_scan_timestamp', 'shipment_method_id', 'shipping_fee',
       'carrier_min_estimate', 'carrier_max_estimate', 'item_zip', 'buyer_zip',
       'category_id', 'item_price', 'quantity', 'payment_datetime',
       'delivery_date', 'weight', 'weight_units', 'package_size',
       'record_number'],
      dtype='object')


# Setting up pipeline

## Target column

The objective is to estimate the total number of calendar days (after payment) it will take to have a purchased item show up at the buyer’s address. This is equivalent to estimating the delivery date using the formula:

payment date (local time) + delivery calendar days = delivery date (local time).

Participants should provide this delivery date in their submissions.

_Now to generate target column_

### Run the basic preprocessing steps

#### Before parallelizing

In [40]:
training_data = preprocessing.basic_preprocessing(training_data)

Finished parse_datetime_columns
Finished create_delivery_calendar_days
Finished clean_zip_codes


In [41]:
training_data["delivery_calendar_days"].describe()

count    1.500000e+07
mean     4.744132e+00
std      3.097165e+00
min     -3.540000e+02
25%      3.000000e+00
50%      4.000000e+00
75%      5.000000e+00
max      2.140000e+02
Name: delivery_calendar_days, dtype: float64

In [42]:
for col in training_data.columns:
    print(f"{col} : {len(training_data[col].unique())}")

b2c_c2c : 2
seller_id : 1759305
declared_handling_days : 12
acceptance_scan_timestamp : 896695
shipment_method_id : 25
shipping_fee : 7044
carrier_min_estimate : 6
carrier_max_estimate : 6
item_zip : 50940
buyer_zip : 57274
category_id : 33
item_price : 41571
quantity : 147
payment_datetime : 12770984
delivery_date : 767
weight : 1298
weight_units : 2
package_size : 7
record_number : 15000000
delivery_calendar_days : 204
cleaned_item_zip : 36147
cleaned_buyer_zip : 38497
distance_between_pincodes : 11915492


# Saving and loading preprocessed data

In [43]:
preprocessed_file_name = "preprocessed_training_data.csv"
preprocessed_data_folder = "preprocessed_data"

preprocessed_data_folder_path = os.path.join("../data", preprocessed_data_folder)
preprocessed_data_path = os.path.join(preprocessed_data_folder_path, preprocessed_file_name)

In [44]:
if not os.path.isdir(preprocessed_data_folder_path):
    os.mkdir(preprocessed_data_folder_path)

In [45]:
if not os.path.isfile(preprocessed_data_path):
    training_data.to_csv(preprocessed_data_path, index = False)

# Start running from here if no extra preprocessing

In [46]:
training_data = pd.read_csv(preprocessed_data_path)

In [47]:
training_data.shape

(15000000, 23)

# Implementing pipeline

## Arbitrarily choosing how to encode columns

In [7]:
different_types_of_encoding = ["one-hot", "numerical"]

cols_and_encoding = {
    "b2c_c2c"   : "one-hot",
    "seller_id"   : 6365,
    "declared_handling_days"   : "numerical",
    "acceptance_scan_timestamp"   : 9848,
    "shipment_method_id"   : "one-hot",
    "shipping_fee"   : "numerical",
    "carrier_min_estimate"   : "numerical",
    "carrier_max_estimate"   : "numerical",
    "item_zip"   : 4701, # Needs a lot of preprocessing.
    "buyer_zip"   : 6880, # Needs a lot of preprocessing.
    "category_id"   : 'one-hot', # Only has 33 unique values in 15,000,000 rows so one-hot.
    "item_price"   : "numerical",
    "quantity"   : "numerical",
    "payment_datetime"   : 9998,
    "delivery_date"   : 602,
    "weight"   : "numerical",
    "weight_units"   : 1,
    "package_size"   : "one-hot",
    "record_number"   : 10000,
    "delivery_calendar_days"   : "target",
    "cleaned_item_zip" : 30743,
    "cleaned_buyer_zip" : 34876,
    "distance_between_pincodes" : "numerical",
}

#### After parallelizing

In [8]:
cols_to_convert_to_one_hot = [ele for ele in cols_and_encoding.keys() if cols_and_encoding[ele] == "one-hot"]

print("cols_to_convert_to_one_hot : ", cols_to_convert_to_one_hot)

print("training_data.shape before : ", training_data.shape)
training_data, generated_columns = preprocessing.one_hot_encode_columns(df = training_data, columns = cols_to_convert_to_one_hot)
print("training_data.shape after : ", training_data.shape)

cols_to_convert_to_one_hot :  ['b2c_c2c', 'shipment_method_id', 'category_id', 'package_size']
training_data.shape before :  (1000000, 23)
training_data.shape after :  (1000000, 80)


In [9]:
target_column = "delivery_calendar_days"
cols_to_use = [ele for ele in cols_and_encoding.keys() if (type(cols_and_encoding[ele]) == str) and
               (cols_and_encoding[ele] != "target") and
               (cols_and_encoding[ele] != "one-hot")]
cols_to_use = [*cols_to_use, *generated_columns]

## Outlier Handling

In [10]:
columns_to_prune = { # Percentage of outliers to drop in each column both top and bottom.
    "shipping_fee" : 0.01,
    "item_price" : 0.001,
    "distance_between_pincodes" : 0.001,
    "quantity" : 0.01,
    "weight" : 0.01,
    "delivery_calendar_days" : 0.01, 
    # This is obviously problematic. Maybe we should have a model that handles outliers in the target
}

### Dropping outliers rows

In [11]:
def remove_outliers(data, columns_to_prune):# This is fundamentally flawed because the thresholds to be pruned should have been calculated earlier not after pruning one or two coloumns.
    for col in columns_to_prune.keys():
        print("col : ", col)
        min_and_max = data[col].quantile([columns_to_prune[col], (1-columns_to_prune[col])]).values
        all_rows_to_drop = np.append(np.where(data[col] > min_and_max[1])[0], np.where(data[col] < min_and_max[0])[0])
#         print("all_rows_to_drop : ", all_rows_to_drop)
        print("all_rows_to_drop.shape : ", all_rows_to_drop.shape)
#         return all_rows_to_drop
        data.drop(all_rows_to_drop, inplace=True)
        print("After dropping rows shape is :", data.shape)
        data.reset_index(inplace = True, drop = True)
#         data[col] = np.where(data[col] > upper_limit, upper_limit, data[col])
#         data[col] = np.where(data[col] < lower_limit, lower_limit, data[col])
    return data

In [12]:
training_data = remove_outliers(data = training_data, columns_to_prune=columns_to_prune)

### Squeezing the outliers

In [13]:
columns_to_squeeze = {
    "shipping_fee",
    "item_price",
    "distance_between_pincodes",
    "quantity",
    "weight",
}

In [14]:
def squeeze_outlier_with_interquantile_range(data, columns):
    for col in columns:
        sorted(data[col])
        Q1, Q3 = data[col].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lower_limit = Q1 - 1.5 * IQR
        upper_limit = Q3 + 1.5 * IQR
        data[col] = np.where(data[col] > upper_limit, upper_limit, data[col])
        data[col] = np.where(data[col] < lower_limit, lower_limit, data[col])
    return data

In [16]:
# backup_training_data = training_data.copy()

# training_data = squeeze_outlier_with_interquantile_range(data = training_data, columns=columns_to_squeeze)

# backup_training_data["shipping_fee"].describe()

In [17]:
training_data["shipping_fee"].describe()

count    1000000.000000
mean           2.361428
std            3.382954
min           -0.250000
25%            0.000000
50%            0.000000
75%            4.000000
max           10.000000
Name: shipping_fee, dtype: float64

### Results and observations

Choosing a custom percentage to crop off for each column will almost certainlycome in handy.

The obvious problem is to detect outliers in delivery_calendar_days.

Loss is reduced after we drop outliers from all the column not only delivery_calendar_days.

Not sure how to deal with that yet. But one thing to keep in mind is that since the evaluation is not a squared metric, we can be allowed to have a small section of the data be wildly off. If the evaluation was squared then this small section would have had a huge impact on the loss. 

## Running a gradient boosted decision tree model

In [19]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn import metrics

### Custom evaluation function

In [20]:
def customLossFunctionEval(y_true, y_pred):
    residual = (y_true - y_pred).astype(np.float32)
    loss = np.where(residual < 0, 0.4 * (residual), 0.6 * (residual))
#     return np.mean(loss)
    return "Custom Loss Function", np.mean(loss), False

In [7]:
print("FDS")

FDS
CPU times: user 105 µs, sys: 0 ns, total: 105 µs
Wall time: 79.9 µs


In [21]:
# To define the input and output feature
# x = training_data.drop(['Embarked','PassengerId'],axis=1)
x = training_data[cols_to_use]
y = training_data[target_column]
# train and test split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((670000, 69), (330000, 69), (670000,), (330000,))

In [22]:
model = lgb.LGBMRegressor(learning_rate=0.1,max_depth=-5,random_state=42)
model.fit(x_train,y_train,eval_set=(x_test,y_test),
          verbose=10,eval_metric=customLossFunctionEval)# This loss needs to mirror the loss that Ebay is using.

/home/swagget/anaconda3/envs/py37/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[10]	valid_0's l2: 8.60008	valid_0's Custom Loss Function: 0.16001
[20]	valid_0's l2: 8.42853	valid_0's Custom Loss Function: 0.157311
[30]	valid_0's l2: 8.38101	valid_0's Custom Loss Function: 0.156484
[40]	valid_0's l2: 8.36465	valid_0's Custom Loss Function: 0.156123
[50]	valid_0's l2: 8.35207	valid_0's Custom Loss Function: 0.155838
[60]	valid_0's l2: 8.34854	valid_0's Custom Loss Function: 0.155776
[70]	valid_0's l2: 8.34368	valid_0's Custom Loss Function: 0.155601
[80]	valid_0's l2: 8.34119	valid_0's Custom Loss Function: 0.15557
[90]	valid_0's l2: 8.33746	valid_0's Custom Loss Function: 0.155502
[100]	valid_0's l2: 8.33811	valid_0's Custom Loss Function: 0.155442


LGBMRegressor(max_depth=-5, random_state=42)

# Evaluating the model

In [ ]:
model_test_outputs = model.predict(x_test)

In [ ]:
pd.Series(model_test_outputs).describe()

In [ ]:
customLossFunctionEval(y_true = y_test, y_pred = model_test_outputs)

In [ ]:
lgb.plot_importance(model)

In [ ]:
feature_importances = {}

for importance, feature in zip(model.feature_importances_, model.feature_name_):
    feature_importances[feature] = importance

feature_importances

In [ ]:
print('Training accuracy {:.4f}'.format(model.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(x_test,y_test)))

In [ ]:
# training_data["model_outputs"] = model.predict(training_data[cols_to_use])

In [ ]:
modes_dict = {}
for col in training_data:
    modes_dict[col] = training_data[col].mode()

In [ ]:
raise

In [ ]:
del training_data
quiz_data = preprocessing.read_data(rows_to_read = 10000)["quiz"]

quiz_data = preprocessing.parse_datetime_columns(quiz_data)

quiz_data, generated_columns = preprocessing.one_hot_encode_columns(df = quiz_data, columns = cols_to_convert_to_one_hot)


In [ ]:
# Filling in the blanks from non existing 

for col in model.feature_name_:
    if col not in quiz_data.columns:
        print("col : ", col)
        quiz_data[col] = modes_dict[col]

In [ ]:
quiz_data["model_outputs"] = model.predict(quiz_data[cols_to_use])

# Postprocessing

After this is all done and predicting with a decent accuracy this needs to be put back and the output needs to predict the delivery date.

The submission is a tsv table, which can be gzipped, with no headeron thw quiz dataset for now. After we win the first phase the test dataset will be given to us.

Each line should contain two values, first the record identifier from the distributed dataset, which is an integer, and second the predicted delivery date as a string in the format YYYY-MM-DD.

All records must be predicted.

In [ ]:
output_columns_name = "predicted_delivery_date"

In [ ]:
postprocessing.generate_output_column(df = quiz_data, 
                                      predicted_days_column="model_outputs",
                                     output_columns_name = output_columns_name)

In [ ]:
postprocessing.generate_submission_file(df = quiz_data, predicted_dates_column="predicted_delivery_date")

# Double checking outputs

In [ ]:
import pandas as pd

In [ ]:
submission = pd.read_csv("./results/result_2021-12-19 00:30:37.tsv", sep='\t', header = None)

In [ ]:
submission